## Install Detectron2


In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# importing libraries


In [ ]:
import pandas as pd
import numpy as np
import pandas as pd 
from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm # progress bar
from datetime import datetime
import time
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import os, json, cv2, random
import skimage.io as io
import copy
from pathlib import Path
from typing import Optional

from tqdm import tqdm
import itertools

import torch
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

from glob import glob
import numba
from numba import jit

import warnings
warnings.filterwarnings('ignore') #Ignore "future" warnings and Data-Frame-Slicing warnings.

# detectron2
from detectron2.structures import BoxMode
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.engine import DefaultPredictor, DefaultTrainer, launch
from detectron2.evaluation import COCOEvaluator
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import Visualizer

from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data import detection_utils as utils

from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

setup_logger()

# Loading Dataset

In [ ]:
!gdown 1fcXLegUtSsdsx6aTT-3OuksdsdsadsaStVV

In [ ]:
import zipfile
from tqdm import tqdm

def extract_with_progress(zip_path, extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Get list of files
        file_list = zip_ref.namelist()
        # Show progress bar
        for file in tqdm(file_list):
            zip_ref.extract(file, extract_path)

# Use the function
extract_with_progress('Exterior.zip', 'data')

In [ ]:
ANNOTATIONS_FILE_PATH = "/kaggle/working/data/annotations/instances_default.json"
IMAGES_DIR_PATH = "/kaggle/working/data/images"

In [ ]:
Data_Resister_training="my_train";
Data_Resister_valid="my_valid";
from detectron2.data.datasets import register_coco_instances
dataDir=Path(IMAGES_DIR_PATH)

register_coco_instances(Data_Resister_training,{}, ANNOTATIONS_FILE_PATH, dataDir)
register_coco_instances(Data_Resister_valid,{}, ANNOTATIONS_FILE_PATH, dataDir)

metadata = MetadataCatalog.get(Data_Resister_training)
dataset_train = DatasetCatalog.get(Data_Resister_training)
dataset_valid = DatasetCatalog.get(Data_Resister_valid)

# Evaluator

In [ ]:
# Taken from https://www.kaggle.com/theoviel/competition-metric-map-iou
from detectron2.evaluation.evaluator import DatasetEvaluator
import pycocotools.mask as mask_util

def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    return np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)

def score(pred, targ):
    pred_masks = pred['instances'].pred_masks.cpu().numpy()
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    
    # Convert segmentation to RLE if it isn't already
    enc_targs = []
    for t in targ:
        if isinstance(t['segmentation'], dict) and 'counts' in t['segmentation']:
            # Already in RLE format
            enc_targs.append(t['segmentation'])
        else:
            # Convert polygon or other format to RLE
            h, w = pred_masks[0].shape  # assuming all masks have same dimensions
            rles = mask_util.frPyObjects(t['segmentation'], h, w)
            enc_targs.append(mask_util.merge(rles))
    
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, ious)
        p = tp / (tp + fp + fn)
        prec.append(p)
    return np.mean(prec)
    
class MAPIOUEvaluator(DatasetEvaluator):
    def __init__(self, dataset_name):
        dataset_dicts = DatasetCatalog.get(dataset_name)
        self.annotations_cache = {item['image_id']:item['annotations'] for item in dataset_dicts}
            
    def reset(self):
        self.scores = []

    def process(self, inputs, outputs):
        for inp, out in zip(inputs, outputs):
            if len(out['instances']) == 0:
                self.scores.append(0)    
            else:
                targ = self.annotations_cache[inp['image_id']]
                self.scores.append(score(out, targ))

    def evaluate(self):
        return {"MaP IoU": np.mean(self.scores)}

class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return MAPIOUEvaluator(dataset_name)
    


# Training

In [ ]:
cfg = get_cfg()
config_name = "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"
cfg.merge_from_file(model_zoo.get_config_file(config_name))

# Dataset Configuration
cfg.DATASETS.TRAIN = (Data_Resister_training,)
cfg.DATASETS.TEST = (Data_Resister_valid,)
cfg.DATALOADER.NUM_WORKERS = 4
cfg.INPUT.MASK_FORMAT = 'bitmask'

# Model Configuration
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(config_name)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 1024
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 29
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3
cfg.MODEL.BACKBONE.FREEZE_AT = 1

# Training Configuration
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0005
cfg.SOLVER.MAX_ITER = 20000
cfg.SOLVER.STEPS = (7000, 9000)

# Optimizer Configuration
cfg.SOLVER.MOMENTUM = 0.9
cfg.SOLVER.WEIGHT_DECAY = 0.0005
cfg.SOLVER.GAMMA = 0.1

# Warmup Configuration
cfg.SOLVER.WARMUP_FACTOR = 1.0 / 2000
cfg.SOLVER.WARMUP_ITERS = 2000
cfg.SOLVER.WARMUP_METHOD = "linear"

# Evaluation and Checkpoint Configuration
cfg.TEST.EVAL_PERIOD = 500
cfg.SOLVER.CHECKPOINT_PERIOD = 500

# Create output directory
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Initialize and start training
trainer = Trainer(cfg)  # or AugTrainer(cfg) for data augmentation
trainer.resume_or_load(resume=False)
trainer.train()

# Evaluator
* Famouns dataset's evaluator is already implemented in detectron2.
* For example, many kinds of AP (Average Precision) are calculted in COCOEvaluator.
* **COCOEvaluator calculates AP with IoU from 0.50 to 0.95**

In [ ]:
evaluator = COCOEvaluator(Data_Resister_valid, cfg, False, output_dir="./output/")
cfg.MODEL.WEIGHTS="../input/detectron2cell/output/model_final.pth"
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2   # set a custom testing threshold
#cfg.INPUT.MASK_FORMAT='polygon'
val_loader = build_detection_test_loader(cfg, Data_Resister_valid)
inference_on_dataset(trainer.model, val_loader, evaluator)

In [ ]:
import pandas as pd
metrics_df = pd.read_json("./output/metrics.json", orient="records", lines=True)
mdf = metrics_df.sort_values("iteration")

# Loss curve

In [ ]:
# 1. Loss curve
fig, ax = plt.subplots()

mdf1 = mdf[~mdf["total_loss"].isna()]
ax.plot(mdf1["iteration"], mdf1["total_loss"], c="C0", label="train")
if "validation_loss" in mdf.columns:
    mdf2 = mdf[~mdf["validation_loss"].isna()]
    ax.plot(mdf2["iteration"], mdf2["validation_loss"], c="C1", label="validation")

# ax.set_ylim([0, 0.5])
ax.legend()
ax.set_title("Loss curve")
plt.show()

# Accuracy curve

In [ ]:
# 1. Accuracy curve
fig, ax = plt.subplots()

mdf1 = mdf[~mdf["fast_rcnn/cls_accuracy"].isna()]
ax.plot(mdf1["iteration"], mdf1["fast_rcnn/cls_accuracy"], c="C0", label="train")
# ax.set_ylim([0, 0.5])
ax.legend()
ax.set_title("Accuracy curve")
plt.show()

# Predictor

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
#cfg.MODEL.WEIGHTS = "./output/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold for this model
cfg.DATASETS.TEST = (Data_Resister_valid, )
predictor = DefaultPredictor(cfg)

In [ ]:
fig, ax = plt.subplots(4, 1, figsize =(20,50))
indices=[ax[0],ax[1],ax[2],ax[3] ]
i=-1
for d in random.sample(dataset_valid, 4):
    i=i+1    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=metadata, 
                   scale=1, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    indices[i].grid(False)
    indices[i].imshow(out.get_image()[:, :, ::-1])

# Remove Folder

In [ ]:
# import shutil

# # Path to the folder to be removed
# folder_to_remove = "/kaggle/working/output"

# # Remove the folder
# shutil.rmtree(folder_to_remove)

# print(f"Folder {folder_to_remove} removed successfully.")


# Remove File

In [ ]:
# import os

# # Path to the file to be removed
# file_to_remove = "/kaggle/working/output"

# # Remove the file
# if os.path.exists(file_to_remove):
#     os.remove(file_to_remove)
#     print(f"File {file_to_remove} removed successfully.")
# else:
#     print(f"File {file_to_remove} does not exist.")
